In [140]:
# Load libraries
import numpy as np
import scipy as sp
import pandas as pd
from pandas import read_csv
%pylab
#%matplotlib inline

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


C:\Users\vmatic\AppData\Local\Continuum\Anaconda3_ticma\lib\site-packages\IPython\core\magics\pylab.py:161: UserWarning: pylab import has clobbered these variables: ['f', 'correlate']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [141]:
# Load .csv files
# these files are created after the face has been detected
# it stores lip positions for 2 (or more) speakers. 


id_file = 5
a1 = read_csv('C:/Users/vmatic/Dropbox/Upwork/Database/database_002/lips'+ str(id_file) +'_1.csv', header =0)
a2 = read_csv('C:/Users/vmatic/Dropbox/Upwork/Database/database_002/lips'+ str(id_file) +'_2.csv', header =0)
print(a1.shape[0])
print(a2.shape[0])


202
202


In [142]:
frame_max = max( a1['Frame'].iloc[-1], a2['Frame'].iloc[-1])
print(frame_max)

202


In [143]:
# it may happen that face detection algorithm fails to detect a face
# for this reason we have developed an extrapolation of the 'lip signal'. 
# this is the only way to solve it since we cannot change face detection algorith code

#X1 - are x postitions of lips for the first speaker
#Y1 - are y postitoins of lips for the first speaker

# w1 shows how large the "head" is, that is, we experimented how it affects the 'lip signal'
# e.g. if one person is very close to the camera, the signals can be different as for someone who is rather far 
# (face will be smaller)

X1 = np.zeros((frame_max,24))
frame_ind = a1['Frame'].values
frame_ind = frame_ind-1
X1[frame_ind,:] = a1.iloc[:,1:25]

Y1 = np.zeros((frame_max,24))
frame_ind = a1['Frame'].values
frame_ind = frame_ind-1
Y1[frame_ind,:] = a1.iloc[:,25:49]

X2 = np.zeros((frame_max,24))
frame_ind = a2['Frame'].values
frame_ind = frame_ind-1
X2[frame_ind,:] = a2.iloc[:,1:25]

Y2 = np.zeros((frame_max,24))
frame_ind = a2['Frame'].values
frame_ind = frame_ind-1
Y2[frame_ind,:] = a2.iloc[:,25:49]

W1 = np.zeros((frame_max))
frame_ind = a1['Frame'].values
frame_ind = frame_ind-1
W1[frame_ind] = a1.iloc[:,-1]

W2 = np.zeros((frame_max))
frame_ind = a2['Frame'].values
frame_ind = frame_ind-1
W2[frame_ind] = a2.iloc[:,-1]



In [144]:
# here we have optimized, what 'lip points' are optimal to be used for further processing

index_dots = np.array([1,2,3,4,5,6,14,15,16,17,18])
index_dots = index_dots - 1

#X1 = a1.iloc[:,1:25]
#X1 = X1.as_matrix()
X1 = X1[:,index_dots]
#Y1 = a1.iloc[:,25:49]
#Y1 = Y1.as_matrix()
Y1 = Y1[:, index_dots]

#X2 = a2.iloc[:,1:25]
#X2 = X2.as_matrix()
X2 = X2[:,index_dots]
#Y2 = a2.iloc[:,25:49]
#Y2 = Y2.as_matrix()
Y2 = Y2[:,index_dots]
#W2 = a2.iloc[:,-1]

In [145]:
# here we perform an interpolation of the "missed frames" due to fails of the algorithm for face detection

from scipy.interpolate import interp1d
for i in range((X1.shape[1])):
    temp = X1[:,i]
    ind_temp = np.arange(len(temp))
    idx    = np.where(temp!=0)
    f = interp1d(ind_temp[idx], temp[idx])
    X1[:,i] = f(ind_temp)
   
    temp = Y1[:,i]
    ind_temp = np.arange(len(temp))
    idx    = np.where(temp!=0)
    f = interp1d(ind_temp[idx], temp[idx])
    Y1[:,i] = f(ind_temp)
    
    temp = X2[:,i]
    ind_temp = np.arange(len(temp))
    idx    = np.where(temp!=0)
    f = interp1d(ind_temp[idx], temp[idx])
    X2[:,i] = f(ind_temp)
    
    temp = Y2[:,i]
    ind_temp = np.arange(len(temp))
    idx    = np.where(temp!=0)
    f = interp1d(ind_temp[idx], temp[idx])
    Y2[:,i] = f(ind_temp)
    
temp = W1
ind_temp = np.arange(len(temp))
idx    = np.where(temp!=0)
f = interp1d(ind_temp[idx], temp[idx])
W1 = f(ind_temp)
    
temp = W2
ind_temp = np.arange(len(temp))
idx    = np.where(temp!=0)
f = interp1d(ind_temp[idx], temp[idx])
W2 = f(ind_temp)
    
    


In [146]:
#pyplot.plot(Y2)

In [147]:
# We assume that lips are forming a convex polygon 
# to calculate area of the lips, we split it into triangles 
# we sum up all triangles and calculate area of the 'mouth'


def area(X, Y):
    n = len(X)
    A = 0
    for i in range(len(X)-2) :
        temp = area_triangle( X[i], Y[i], X[i+1], Y[i+1], X[i+2], Y[i+2])
        A = A + temp
    
    return A

def area_triangle(x1, y1, x2, y2, x3, y3):
    a = np.sqrt( (x1-x2)**2  +  (y1 - y2)**2 )
    b = np.sqrt( (x1-x3)**2  +  (y1 - y3)**2 )
    c = np.sqrt( (x3-x2)**2  +  (y3 - y2)**2 )
    s = (a+b+c) / 2
    area = np.sqrt (s * (s-a) * (s-b) * (s-c) )
    
    return area
    
    

In [148]:
# for each video frame we calculate mouth area 
# thus we obtain the mouth area signal for each frame

def area_signal(X, Y):
    signal1 = np.zeros(X.shape[0])
    for i in range(X.shape[0]):
        signal1[i] = area(X[i,:], Y[i,:])
        
    
    return signal1


In [149]:
#W1 = 1.0 / W1
#W2 = 1.0 / W2

# mouth area signals are created for a set of points - that is their, X and Y coordinates
x_area1 = area_signal(X1, Y1) #* W1 * W1
x_area2 = area_signal(X2, Y2) #* W2 * W2



C:\Users\vmatic\AppData\Local\Continuum\Anaconda3_ticma\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in sqrt


In [150]:
# we load autio file

import scipy.io.wavfile 
Audio = scipy.io.wavfile.read('C:/Users/vmatic/Dropbox/Upwork/Database/database_002/lips'+ str(id_file)+ '_audio.wav')
Audio = Audio[1][:,0]

In [151]:
# basic audio processing

Audio = Audio - Audio.mean()
Audio = Audio / Audio.std() 
Fs =  44100
t = np.arange(0, len(Audio))
t = t / Fs

In [152]:
# we calculate in this way the frame rate
frame_rate = X1.shape[0] / Audio.shape[0] * 44100  

In [153]:
frame_rate

7.662901824500434

In [154]:
Audio.shape[0]

1162510

In [155]:
# we calculate hilbert transform and calculate Audio envelope
from scipy.signal import hilbert
analytic_signal = hilbert(Audio)
amplitude_envelope = np.abs(analytic_signal)
instantaneous_phase = np.unwrap(np.angle(analytic_signal))
fs = 44100
instantaneous_frequency = (np.diff(instantaneous_phase) / (2.0*np.pi) * fs)

In [156]:
# we resample audio signal to the frame rate
# only in this way we are able to correlate the mouth area signal and 'audio envelope'
from scipy import signal
Fs = 44100
x_audio = signal.resample(amplitude_envelope,  round( len(amplitude_envelope) / Fs  * frame_rate)   )


In [157]:
pyplot.plot(x_area1, 'b')
pyplot.plot(x_area2, 'k')




In [158]:
# we have processed all three signals -  bandpassed them to better capture the common dynamics

x_area1 = x_area1 - x_area1[0]
x_area2 = x_area2 - x_area2[0]
b, a = sp.signal.butter(6, [0.1, 0.5], btype='bandpass')
x1 = sp.signal.lfilter(b, a, x_area1)
x2 = sp.signal.lfilter(b, a, x_area2)
x0 = sp.signal.lfilter(b, a, x_audio)




pyplot.plot(x_area2)
pyplot.plot(x2,'r')

In [159]:
from scipy.signal import correlate

In [160]:
#here we compare 2 mouth area signals 
# x1 and x2 
# with audio envelope x0

# these signals segments are compared within the sliding window that slides through signals
# window is the number of samples used for comparison
# slide is a step that we use to move the signal
# we store maximum of cross correlation function in C1 variable


window = 20
slide = 2
duzina  = int(np.ceil  ((len(x0)-window) / slide))

C1 = np.zeros( (duzina,2))
for i in range (duzina):
    index_begin = i*slide  
    index_end   = (i+1) * slide + window
    temp1= correlate(x0[index_begin:index_end] , x1[index_begin:index_end] )
    temp11 = np.max(np.absolute(temp1))
    C1[i,0]  = temp11
    temp2= correlate(x0[index_begin:index_end] , x2[index_begin:index_end] )
    temp21 = np.max(np.absolute(temp2))
    C1[i,1]  = temp21
    


In [161]:
t_index = np.arange(0+int(window/2),int(window/2)+duzina*slide, slide) / frame_rate
pyplot.subplot(311)
pyplot.plot(np.arange(0,len(x0      )) / frame_rate , x0,'r')
pyplot.ylabel('Audio envelope')

pyplot.xlim( 0,30  )
pyplot.subplot(312)

pyplot.plot(np.arange(0,len(x0      )) / frame_rate , x1,'b')
pyplot.plot(np.arange(0,len(x0      )) / frame_rate , x2,'k')
pyplot.ylabel('Mouth area signals')

pyplot.xlim(0, 30)
pyplot.subplot(313)

pyplot.plot(t_index, C1[:,0], 'b')
pyplot.plot(t_index, C1[:,1], 'k')
pyplot.xlabel('Two cross-correlation fs for every window: max value determines the speaker')

pyplot.xlim(0, 30)


(0, 30)